## Import Libraries

# TO DO:
- Criar funções para as tarefas
- Bloco while para atualizar status da criação do Redshift;
- Automatizar a criação da infra pelo CLI com biblioteca argparser (Argumentos: create - cria a infra, delete - deleta infra)

In [120]:
# Import libraries

import os
import json
import boto3
import configparser
import pandas as pd
import psycopg2

## Configuration Variables

In [121]:
project_dir = os.path.dirname(os.getcwd())
config_dir = project_dir + "\config\dwh.cfg"

In [122]:
# Configuration variables
config = configparser.ConfigParser()
config.read_file(open(config_dir))

# AWS Credentials
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

#Redshift Cluster
DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

## Step 1: Create IAM, EC2, S3 and Redshift Client

In [123]:
# IAM - Web service for securely controlling access to Amazon Web Services services.
iam = boto3.client("iam",
                   region_name="us-west-2",
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

# EC2
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )
# S3
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

# Redshift
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Step 2: IAM

In [124]:
# Create IAM Role and Attach Policy to Role

try:
    print(("="*15) + " STEP 2.1: Creating IAM Role " + ("="*15))
    dwhRole = iam.create_role(
        Path="/",
        RoleName=DWH_IAM_ROLE_NAME,
        Description="Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps({
            "Statement": [{"Action": "sts:AssumeRole",
            "Effect": "Allow",
            "Principal": {"Service": "redshift.amazonaws.com"}}],
            "Version": "2012-10-17"})
        )
    print("IAM Role Created successfully!")
except Exception as e:
    print("Failed to create IAM Role! Exception: ",e)

try:
    print("Attaching Policy to Role...")

    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                           PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']
    print("Attached Policy to IAM Role")
except Exception as e:
    print("Failed to create IAM Role! Exception: ",e)

print("Getting IAM Role...")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

=============== STEP 2.1: Creating IAM Role ===============
IAM Role Created successfully!
Attaching Policy to Role...
Attached Policy to IAM Role
Getting IAM Role...
arn:aws:iam::524095156763:role/dwhRole


## Step 3.1: Redshift Cluster

In [125]:
# Create Redshift Cluster

try:
    print(("="*15) + " STEP 3.1: Creating Redshift Cluster " + ("="*15))
    response = redshift.create_cluster(        
        #DHW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print("Failed to create Redshift Cluster! Exception: ", e)

=============== STEP 3.1: Creating Redshift Cluster ===============


### Step 3.2 - Describe Redshift Status Creation

In [133]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    print(props["ClusterStatus"]) #available
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

available


C:\Users\Pedrosa\AppData\Local\Temp\ipykernel_16600\2854007945.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.crozrlg6bgbr.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-09a8d81a36277b8a8
7,NumberOfNodes,4


### Step 3.3 - Get DWH Endpoint and Role ARN
<font color='red'>**Wait to run until ClusterStatus equals to Available**</font>

In [134]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.crozrlg6bgbr.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::524095156763:role/dwhRole


In [135]:
config.set("IAM_ROLE", "ARN", DWH_ROLE_ARN)
config.set("CLUSTER", "HOST", DWH_ENDPOINT)
config.set("CLUSTER", "DB_NAME", DWH_DB)
config.set("CLUSTER", "DB_USER", DWH_DB_USER)
config.set("CLUSTER", "DB_PASSWORD", DWH_DB_PASSWORD)
config.set("CLUSTER", "DB_PORT", DWH_PORT)

with open(config_dir, "w") as configfile:
    config.write(configfile)

## Step 4 - Open an incoming TCP port 5439 to access the cluster endpoin

In [136]:
try:
    print(("="*15) + " STEP 4: Opening Port 5439 of Redshift Cluster " + ("="*15))
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print("Failed to open port 5439! Exception: ", e)

=============== STEP 4: Opening Port 5439 of Redshift Cluster ===============
ec2.SecurityGroup(id='sg-041896f01c53cde4d')


## Step 5 - Connect to Redshift Cluster

In [100]:
try:
    print(("="*15) + " STEP 5: Connecting to Redshift " + ("="*15))
    conn = psycopg2.connect(host=DWH_ENDPOINT, database=DWH_DB, user=DWH_DB_USER, password=DWH_DB_PASSWORD, port=DWH_PORT)
    print("Connected to Redshift!")
except Exception as e:
    print("Failed to connect to Redshift! Exception: ", e)

=============== STEP 5: Connecting to Redshift ===============
Connected to Redshift!


In [110]:
cursor = conn.cursor()

In [112]:
cursor.execute("SELECT * FROM staging_events LIMIT 5;")
results = cursor.fetchall()
for result in results:
    print(result)

## Step 6 -  Delete resources
- run this block several times until the cluster really deleted

In [139]:
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

InvalidClusterStateFault: An error occurred (InvalidClusterState) when calling the DeleteCluster operation: There is an operation running on the Cluster. Please try to delete it at a later time.

In [173]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.

In [170]:
#### CAREFUL!!
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: The role with name dwhRole cannot be found.